In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/embeddings"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
df['question_text'][1306117]

In [ ]:
df.shape

In [ ]:
df[df['question_text'].isnull()]

In [ ]:
df[df['target']==1].shape

In [ ]:
df[df['target']==0]

In [ ]:
df['question_text'].str.split().str.len().max()

In [ ]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features,split=' ')


In [ ]:
tokenizer.fit_on_texts(df['question_text'].values)
x = tokenizer.texts_to_sequences(df['question_text'].values)
x

In [ ]:
len(x)

In [ ]:
X_train = sequence.pad_sequences(x)
X_train.shape

In [ ]:
y_train = df['target'].values
y_train.shape

In [ ]:
model = Sequential()
model.add(Embedding(max_features,128,input_length=X_train.shape[1]))
model.add(LSTM(256,recurrent_dropout=0.25))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,batch_size=1500,epochs=5,validation_split=0.40,verbose=1)

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.head()

In [ ]:
testx = tokenizer.texts_to_sequences(test_df['question_text'].values)
X_test = sequence.pad_sequences(testx,maxlen=135)

all_preds = []
y_pred = model.predict(X_test,batch_size=128,verbose=1)
all_preds.append(y_pred.flatten())

In [ ]:
print(y_pred.shape)
y_te = (np.array(y_pred.flatten()) > 0.35).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)